In [1]:
import boto3
import pickle


def get_creds():
    sm_client = boto3.client(
        'secretsmanager',
        region_name='us-east-1'
    )

    secret_token = sm_client.get_secret_value(SecretId='gmail_token')['SecretBinary']
    creds = pickle.loads(secret_token)
    return creds

In [2]:
from googleapiclient.discovery import build

creds = get_creds()
service = build('gmail', 'v1', credentials=creds)

In [3]:
users = service.users()

In [4]:
dynamodb = boto3.resource('dynamodb')

In [5]:
table = dynamodb.Table('gmail_jobs')

In [6]:
item = {
    'job_id': 'gmail_ingest',
    'job_description': 'Ingest data from gmail to s3',
    'is_active': 'Y',
    'baseline_days': 45
}

In [7]:
table.put_item(Item=item)

{'ResponseMetadata': {'RequestId': '9O6IGRQJ29OL6SHNHLAU95TQAFVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sat, 14 May 2022 08:02:01 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '9O6IGRQJ29OL6SHNHLAU95TQAFVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [8]:
def get_job_details(job_name):
    dynamodb = boto3.resource('dynamodb')
    table = dynamodb.Table('gmail_jobs')
    job_details = table.get_item(Key={'job_id': job_name})['Item']
    return job_details

In [9]:
import datetime
import time

def get_job_run_time_range(job_details):
    if job_details.get('job_run_bookmark_details'):
        job_run_bookmark_details = job_details.get('job_run_bookmark_details')
        last_run_start_time_epoch = int(job_run_bookmark_details['last_run_start_time_epoch'])
        last_run_end_time_epoch = int(job_run_bookmark_details['last_run_end_time_epoch'])
        last_run_diff = datetime.datetime.now().date() - datetime.datetime.fromtimestamp(last_run_end_time_epoch).date()
        if last_run_diff.days > 1:
            start_time_epoch = last_run_end_time_epoch
            end_time = datetime.datetime.fromtimestamp(last_run_end_time_epoch).date() + datetime.timedelta(days=1)
            end_time_epoch = int(time.mktime(end_time.timetuple()))
        else:
            start_time_epoch = last_run_end_time_epoch
            end_time_epoch = int(time.mktime(datetime.datetime.now().timetuple()))  
    else:
        baseline_days = int(job_details['baseline_days'])
        start_time = datetime.datetime.now().date() - datetime.timedelta(days=int(baseline_days))
        end_time = start_time + datetime.timedelta(days=1)
        start_time_epoch = int(time.mktime(start_time.timetuple()))
        end_time_epoch = int(time.mktime(end_time.timetuple()))
    return start_time_epoch, end_time_epoch

In [10]:
job_details = get_job_details('gmail_ingest')

In [11]:
get_job_run_time_range(job_details)

(1648578600, 1648665000)

In [12]:
job_details = {
    'job_id': 'gmail_ingest',
    'job_description': 'Ingest data from gmail to s3',
    'is_active': 'Y',
    'baseline_days': 45,
    'job_run_bookmark_details': {
        'last_run_message_id': '17e144500cc9efec',
        'last_run_start_time_epoch': int(time.mktime(datetime.datetime.now().date().timetuple())),
        'last_run_end_time_epoch': int(time.mktime((datetime.datetime.now() - datetime.timedelta(hours=2, minutes=10)).timetuple()))
    }
}

In [13]:
job_details

{'job_id': 'gmail_ingest',
 'job_description': 'Ingest data from gmail to s3',
 'is_active': 'Y',
 'baseline_days': 45,
 'job_run_bookmark_details': {'last_run_message_id': '17e144500cc9efec',
  'last_run_start_time_epoch': 1652466600,
  'last_run_end_time_epoch': 1652507531}}

In [14]:
get_job_run_time_range(job_details)

(1652507531, 1652515334)